In [1]:
import requests
from IPython.display import HTML, JSON

### Set URL of the Flask application

In [2]:
server_url = "http://127.0.0.1:5000"

### Get some info on the `demo` dataset rendered as html (not part of the REST API)

In [3]:
response = requests.get(server_url + "/datasets/demo")

HTML(response.text)

### Get all field names in the `demo` dataset

In [4]:
response = requests.get(server_url + "/api/v1.0/datasets/demo/fieldnames")

response.json()

{'fieldnames': ['air']}

In [5]:
JSON(response.json(), expanded=True)

<IPython.core.display.JSON object>

### Print detailed information about the `demo` dataset

In [6]:
response = requests.get(server_url + "/api/v1.0/datasets/demo/info")

print(response.json()["info"])

xarray.Dataset {
dimensions:
	lat = 25 ;
	lon = 53 ;
	time = 2920 ;

variables:
	float32 lat(lat) ;
		lat:standard_name = latitude ;
		lat:long_name = Latitude ;
		lat:units = degrees_north ;
		lat:axis = Y ;
	float32 air(time, lat, lon) ;
		air:long_name = 4xDaily Air temperature at sigma level 995 ;
		air:units = degK ;
		air:precision = 2 ;
		air:GRIB_id = 11 ;
		air:GRIB_name = TMP ;
		air:var_desc = Air temperature ;
		air:dataset = NMC Reanalysis ;
		air:level_desc = Surface ;
		air:statistic = Individual Obs ;
		air:parent_stat = Other ;
		air:actual_range = [185.16 322.1 ] ;
	float32 lon(lon) ;
		lon:standard_name = longitude ;
		lon:long_name = Longitude ;
		lon:units = degrees_east ;
		lon:axis = X ;
	datetime64[ns] time(time) ;
		time:standard_name = time ;
		time:long_name = Time ;

// global attributes:
	:Conventions = COARDS ;
	:title = 4x daily NMC reanalysis (1948) ;
	:description = Data is from NMC initialized reanalysis
(4x/day).  These are the 0.9950 sigma level valu

In [7]:
JSON(response.json())

<IPython.core.display.JSON object>

### Extract data along one or more ship tracks

Query specs:

- fieldnames: one or more field names (data variables) -- REQUIRED
- tracks: ship tracks formatted as GeoJSON -- REQUIRED
- transform: some parameters to first transform the dataset before extraction -- OPTIONAL

In [8]:
tracks_geojson = {
    "type": "FeatureCollection",
    "features": [{
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": [
                [71.591269, 27.809646],
                [74.112343, 34.536865],
                [77.23488, 60.95901],
                [81.77730, 80.55286]
            ]
        },
        "properties": {
            "track_id": 1,
        }
    }]
}

transform = {
    "aggregation": "mean",
    "dim": "time"
}

query = {
    "fieldnames": "air",
    "transform": transform,
    "tracks": tracks_geojson
}

response = requests.post(server_url + "/api/v1.0/datasets/demo/extract_tracks", json=query)

response.json()

{'data': [{'air': [262.73370361328125,
    260.3756408691406,
    260.3756408691406,
    260.3756408691406],
   'track_id': 1}]}

In [9]:
JSON(response.json(), expanded=True)

<IPython.core.display.JSON object>